**Basic ItemKNN recommender**

In [1]:
%load_ext cython

In [2]:
import os
from typing import Tuple,Callable,Dict,Optional,List

import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import random

from sklearn.model_selection import train_test_split

**Dataset loading with pandas**

The function read_csv from pandas provides a wonderful and fast interface to load tabular data like this. For better results and performance we provide the separator ::, the column names ["user_id", "item_id", "ratings", "timestamp"], and the types of each attribute in the dtype parameter.

In [3]:
def load_data():
  return pd.read_csv("./data_train.csv")

In [4]:
ratings=load_data()
d ={'user_id': ratings['row'],'item_id':ratings['col'],'ratings':ratings['data']}
ratings=pd.DataFrame(data=d)

In [5]:
ratings.dtypes

user_id      int64
item_id      int64
ratings    float64
dtype: object

In [6]:
userList=list(d['user_id'])
itemList=list(d['item_id'])
ratingList=list(d['ratings'])

In [7]:
URM_all = sp.coo_matrix((ratingList,(userList,itemList)))
URM_all = URM_all.tocsr()

In [8]:
URM_all

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113268 stored elements in Compressed Sparse Row format>

In [9]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [10]:
features=load_data()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)

In [11]:
featureList=list(d['feature_id'])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(featureList)

featureList = le.transform(featureList)

In [12]:
ones = np.ones(len(featureList))
ICM_all = sp.coo_matrix((ones, (itemList,featureList)), shape=(URM_all.shape[1],featureList.max()+1))
ICM_all = ICM_all.tocsr()

In [13]:
URM_all

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113268 stored elements in Compressed Sparse Row format>

In [14]:
ICM_all

<25975x24896 sparse matrix of type '<class 'numpy.float64'>'
	with 24896 stored elements in Compressed Sparse Row format>

This section wors with the previously-loaded ratings dataset and extracts the number of users, number of items, and min/max user/item identifiers. Exploring and understanding the data is an essential step prior fitting any recommender/algorithm.

In this specific case, we discover that item identifiers go between 1 and 25974, however, there are only 24896 different items. To ease further calculations, we create new contiguous user/item identifiers, we then assign each user/item only one of these new identifiers. To keep track of these new mappings, we add them into the original dataframe using the pd.merge function.

In [15]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()
    
    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()
    
    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)
    
    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})
    
    ratings = pd.merge(left=ratings, 
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")
    
    ratings = pd.merge(left=ratings, 
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")
    
    return ratings



In [16]:
ratings=preprocess_data(ratings)

7947 0 7946
24896 0 25974


In [17]:
ratings

,user_id,item_id,ratings,mapped_user_id,mapped_item_id
0,0,10080,1.0,0,0
1,4342,10080,1.0,4342,0
2,5526,10080,1.0,5526,0
3,5923,10080,1.0,5923,0
4,0,19467,1.0,0,1
5,149,19467,1.0,149,1
6,4072,19467,1.0,4072,1
7,6193,19467,1.0,6193,1
8,7105,19467,1.0,7105,1
9,1,2665,1.0,1,2


In [18]:
unique_users = ratings.mapped_user_id.unique()
unique_items = ratings.mapped_item_id.unique()
num_users=unique_users.size
num_items=unique_items.size

In [19]:
num_users

7947

In [20]:
num_items

24896

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [21]:
def dataset_splits(ratings, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234
    
    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     ratings_training, ratings_test) = train_test_split(ratings.mapped_user_id,
                                                        ratings.mapped_item_id,
                                                        ratings.ratings,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)
    
    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     ratings_training, ratings_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              ratings_training,
                                                              test_size=validation_percentage,
                                                             )
    
    urm_train = sp.csr_matrix((ratings_training, (user_ids_training, item_ids_training)), 
                              shape=(num_users, num_items))
    
    urm_validation = sp.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)), 
                              shape=(num_users, num_items))
    
    urm_test = sp.csr_matrix((ratings_test, (user_ids_test, item_ids_test)), 
                              shape=(num_users, num_items))
    

    
    return urm_train, urm_validation, urm_test



In [22]:
urm_train, urm_validation, urm_test = dataset_splits(ratings, 
                                                     num_users, 
                                                     num_items, 
                                                     validation_percentage=0.10, 
                                                     testing_percentage=0.20)

**SLIM MSE**

Our objective is to minimize the MSE ( mean square error) by choosing the best similarity matrix.

**Collaborative Filtering ItemKNN Recommender**

This step creates a CFItemKNN class that represents a Collaborative Filtering ItemKNN Recommender. As we have mentioned in previous practice sessions, our recommenders have two main functions: fit and recommend. In addition we added a filter_seen and the train_multiple_epochs_SLIM_MSE.

In [28]:
class CFItemKNN(object):
    def __init__(self):

        self.similarity_matrix = None
        
    def fit(self,urm_train,learning_rate,epochs):
        
        #if not sp.isspmatrix_csc(urm_train):
        #    raise TypeError(f"We expected a CSC matrix, we got {type(urm_train)}")
        
        self.similarity_matrix=self.train_multiple_epochs_SLIM_MSE(urm_train,learning_rate,epochs)
        
        
    def recommend(self,user_id,urm_train: sp.csr_matrix,at=None,remove_seen=True):
        # compute the scores using the dot product
        user_profile = urm_train[user_id]
        scores = user_profile.dot(self.similarity_matrix).ravel()

        if remove_seen:
            scores = self.filter_seen(urm_train,user_id,scores)

        # rank items
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    
    def filter_seen(self,urm_train,user_id,scores):

        start_pos = urm_train.indptr[user_id]
        end_pos = urm_train.indptr[user_id+1]

        user_profile = urm_train.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf

        return scores  
    
    
    def train_multiple_epochs_SLIM_MSE(self,URM_train, learning_rate_input, n_epochs):
    
        URM_train_coo = URM_train.tocoo()
        n_items = URM_train.shape[1]
        n_interactions = URM_train.nnz


        URM_train_coo_row = URM_train_coo.row
        URM_train_coo_col = URM_train_coo.col
        URM_train_coo_data = URM_train_coo.data
        URM_train_indices = URM_train.indices
        URM_train_indptr = URM_train.indptr
        URM_train_data = URM_train.data

        # We create a dense similarity matrix, initialized as zero
        item_item_S = np.zeros((n_items, n_items), dtype = np.float)

        learning_rate = learning_rate_input
        loss = 0.0

        for n_epoch in range(n_epochs):

            loss = 0.0
            start_time = time.time()

            for sample_num in range(n_interactions):

                # Randomly pick sample
                index = random.randrange(0,n_interactions)  #randrange(x,y) x included y not included
                user_id = URM_train_coo_row[index]
                item_id = URM_train_coo_col[index]
                true_rating = URM_train_coo_data[index]

                # Compute prediction
                start_profile = URM_train_indptr[user_id]
                end_profile = URM_train_indptr[user_id+1]
                predicted_rating = 0.0

                for index in range(start_profile, end_profile):
                    profile_item_id = URM_train_indices[index]
                    profile_rating = URM_train_data[index]
                    predicted_rating += item_item_S[profile_item_id,item_id] * profile_rating

                # Compute prediction error, or gradient.
                prediction_error = true_rating - predicted_rating
                loss += prediction_error**2

                # Update model, in this case the similarity
                for index in range(start_profile, end_profile):
                    profile_item_id = URM_train_indices[index]
                    profile_rating = URM_train_data[index]
                    item_item_S[profile_item_id,item_id] += learning_rate * prediction_error * profile_rating

    #             # Print some stats
    #             if (sample_num +1)% 1000000 == 0:
    #                 elapsed_time = time.time() - start_time
    #                 samples_per_second = (sample_num+1)/elapsed_time
    #                 print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num+1), samples_per_second))


            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num+1)/elapsed_time

            print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/(sample_num+1), samples_per_second))
            
        return np.array(item_item_S) 

**Evaluation Metrics**

In this practice session we will be using the same evaluation metrics defined in the Practice session 2, i.e., precision, recall and mean average precision (MAP).

In [29]:

def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [30]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]
        
        relevant_items = urm_test.indices[user_profile_start:user_profile_end]
        
        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
        recommendations = recommender.recommend(user_id=user_id,
                                               urm_train = urm_train,
                                               at=recommendation_length, 
                                               remove_seen=True)
        
        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)
        
        num_users_evaluated += 1
        
    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

### Practice 8 - Hybrid recommenders


### The way to go to achieve the best recommendation quality

## A few info about hybrids


#### There are many different types of hibrids, in this practice we will see the following
* Linear combination of item-based models
* Linear combination of heterogeneous models
* User-wise discrimination


In [27]:
### Step 1: Import the evaluator objects

from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[5])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[5, 10])

In [28]:
evaluator_validation,evaluator_test

(<Base.Evaluation.Evaluator.EvaluatorHoldout at 0x2ba971426d8>,
 <Base.Evaluation.Evaluator.EvaluatorHoldout at 0x2ba96e77198>)

In [29]:
### Step 2: Create BayesianSearch object
import skopt
import scipy.sparse as sps
#from skopt import gp_minimize
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt


recommender_class = ItemKNNCFRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [30]:
### Step 3: Define parameters range

from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine"])
hyperparameters_range_dictionary["normalize"] = Categorical([True, False])
    
    
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


output_folder_path = "result_experiments/"

import os

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    

In [31]:

### Step 4: Run!    --- error

n_cases = 2
metric_to_optimize = "MAP"

parameterSearch.search(recommender_input_args,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = 1,
                       save_model = "no",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

Iteration No: 1 started. Evaluating function at random point.
ItemKNNCFRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNCFRecommender: URM Detected 2076 (8.34 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 609, 'shrink': 578, 'similarity': 'cosine', 'normalize': False}
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24896 ( 100 % ), 2800.76 column/sec, elapsed time 0.15 min
EvaluatorHoldout: Processed 3654 ( 100.00% ) in 2.45 sec. Users per second: 1492
SearchBayesianSkopt: New best config found. Config 0: {'topK': 609, 'shrink': 578, 'similarity': 'cosine', 'normalize': False} - results: ROC_AUC: 0.0600712, PRECISION: 0.0210728, PRECISION_RECALL_MIN_DEN: 0.0573162, RECALL: 0.0553876, MAP: 0.0325347, MRR: 0.0556468, NDCG: 0.0427990, F1: 0.0305301, HIT_RATE: 0.1053640, ARHR: 0.0578407, NOVELTY: 0.0023121, AVERAGE_POPULARITY: 0.1836063, DIVERSITY_MEAN_INTER_LIST: 0.9705516, DIVERSITY_HERFINDAHL: 0.9940572, COVERAGE_ITEM: 0.1611504, C

In [32]:
### Step 4: Run!
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata

best_parameters = search_metadata["hyperparameters_best"]
best_parameters

# Linear combination of item-based models

#### Let's use an ItemKNNCF with the parameters we just learned and a graph based model

itemKNNCF = ItemKNNCFRecommender(urm_train)
itemKNNCF.fit(**best_parameters)

ItemKNNCFRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNCFRecommender: URM Detected 2076 (8.34 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24896 ( 100 % ), 2880.66 column/sec, elapsed time 0.14 min


In [33]:
from GraphBased.P3alphaRecommender import P3alphaRecommender

P3alpha = P3alphaRecommender(urm_train)
P3alpha.fit()

itemKNNCF.W_sparse

P3alpha.W_sparse

P3alphaRecommender: URM Detected 143 (1.80 %) cold users.
P3alphaRecommender: URM Detected 2076 (8.34 %) cold items.


<24896x24896 sparse matrix of type '<class 'numpy.float32'>'
	with 1046288 stored elements in Compressed Sparse Row format>

In [34]:
from Base.Recommender_utils import check_matrix, similarityMatrixTopK
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender


class ItemKNNSimilarityHybridRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNSimilarityHybridRecommender
    Hybrid of two similarities S = S1*alpha + S2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNSimilarityHybridRecommender"


    def __init__(self, URM_train, Similarity_1, Similarity_2, sparse_weights=True):
        super(ItemKNNSimilarityHybridRecommender, self).__init__(urm_train)

        if Similarity_1.shape != Similarity_2.shape:
            raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {}, S2 is {}".format(
                Similarity_1.shape, Similarity_2.shape
            ))

        # CSR is faster during evaluation
        self.Similarity_1 = check_matrix(Similarity_1.copy(), 'csr')
        self.Similarity_2 = check_matrix(Similarity_2.copy(), 'csr')


    def fit(self, topK=100, alpha = 0.5):

        self.topK = topK
        self.alpha = alpha

        W = self.Similarity_1*self.alpha + self.Similarity_2*(1-self.alpha)
        self.W_sparse = similarityMatrixTopK(W, k=self.topK).tocsr()
       

hybridrecommender = ItemKNNSimilarityHybridRecommender(urm_train, itemKNNCF.W_sparse, P3alpha.W_sparse)
hybridrecommender.fit(alpha = 0.5)

evaluator_validation.evaluateRecommender(hybridrecommender)

### In this case the alpha coefficient is too a parameter to be tune

ItemKNNSimilarityHybridRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2076 (8.34 %) cold items.
EvaluatorHoldout: Processed 3654 ( 100.00% ) in 2.24 sec. Users per second: 1630


({5: {'ROC_AUC': 0.057562488596971356,
   'PRECISION': 0.020032840722496014,
   'PRECISION_RECALL_MIN_DEN': 0.05327494982667399,
   'RECALL': 0.05113925505296939,
   'MAP': 0.031164249224594053,
   'MRR': 0.05328407224958948,
   'NDCG': 0.04035084964794152,
   'F1': 0.028788376679962563,
   'HIT_RATE': 0.10016420361247948,
   'ARHR': 0.05527276044517422,
   'NOVELTY': 0.002671600520724747,
   'AVERAGE_POPULARITY': 0.05604018021492563,
   'DIVERSITY_MEAN_INTER_LIST': 0.997042806663619,
   'DIVERSITY_HERFINDAHL': 0.9993539886558402,
   'COVERAGE_ITEM': 0.3088849614395887,
   'COVERAGE_ITEM_CORRECT': 0.012090295629820051,
   'COVERAGE_USER': 0.45979614949037373,
   'COVERAGE_USER_CORRECT': 0.04303510758776897,
   'DIVERSITY_GINI': 0.16760618499174454,
   'SHANNON_ENTROPY': 12.104265680684037}},
 'CUTOFF: 5 - ROC_AUC: 0.0575625, PRECISION: 0.0200328, PRECISION_RECALL_MIN_DEN: 0.0532749, RECALL: 0.0511393, MAP: 0.0311642, MRR: 0.0532841, NDCG: 0.0403508, F1: 0.0287884, HIT_RATE: 0.1001642, 

In [35]:
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender

pureSVD = PureSVDRecommender(urm_train)
pureSVD.fit()

class ItemKNNScoresHybridRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridRecommender"


    def __init__(self, URM_train, Recommender_1, Recommender_2):
        super(ItemKNNScoresHybridRecommender, self).__init__(urm_train)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2
        
        
    def fit(self, alpha = 0.5):

        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights


hybridrecommender = ItemKNNScoresHybridRecommender(urm_train, itemKNNCF, pureSVD)
hybridrecommender.fit(alpha = 0.5)

evaluator_validation.evaluateRecommender(hybridrecommender)

PureSVDRecommender: URM Detected 143 (1.80 %) cold users.
PureSVDRecommender: URM Detected 2076 (8.34 %) cold items.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... Done!
ItemKNNScoresHybridRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNScoresHybridRecommender: URM Detected 2076 (8.34 %) cold items.
EvaluatorHoldout: Processed 3654 ( 100.00% ) in 3.83 sec. Users per second: 955


({5: {'ROC_AUC': 0.054118773946360145,
   'PRECISION': 0.019321291735084938,
   'PRECISION_RECALL_MIN_DEN': 0.0502234993614304,
   'RECALL': 0.04795149149475847,
   'MAP': 0.028793863650185502,
   'MRR': 0.05136380222587122,
   'NDCG': 0.037607580177143385,
   'F1': 0.02754411849253376,
   'HIT_RATE': 0.0966064586754242,
   'ARHR': 0.05391351943076084,
   'NOVELTY': 0.002240175101101491,
   'AVERAGE_POPULARITY': 0.1817397012390264,
   'DIVERSITY_MEAN_INTER_LIST': 0.9714196412932454,
   'DIVERSITY_HERFINDAHL': 0.9942307580538712,
   'COVERAGE_ITEM': 0.09543701799485861,
   'COVERAGE_ITEM_CORRECT': 0.007350578406169666,
   'COVERAGE_USER': 0.45979614949037373,
   'COVERAGE_USER_CORRECT': 0.04077010192525481,
   'DIVERSITY_GINI': 0.020004942530714227,
   'SHANNON_ENTROPY': 8.754445816575611}},
 'CUTOFF: 5 - ROC_AUC: 0.0541188, PRECISION: 0.0193213, PRECISION_RECALL_MIN_DEN: 0.0502235, RECALL: 0.0479515, MAP: 0.0287939, MRR: 0.0513638, NDCG: 0.0376076, F1: 0.0275441, HIT_RATE: 0.0966065, A

In [36]:
# User-wise hybrid   -- ICM missing

### Models do not have the same accuracy for different user types. Let's divide the users according to their profile length and then compare the recommendation quality we get from a CF model



URM_train = sps.csr_matrix(urm_train)

profile_length = np.ediff1d(urm_train.indptr)

### Let's select a few groups of 5% of the users with the least number of interactions

block_size = int(len(profile_length)*0.05)
block_size

sorted_users = np.argsort(profile_length)

for group_id in range(0, 10):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, average p.len {:.2f}, min {}, max {}".format(group_id, 
        users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))

### Now we plot the recommendation quality of TopPop and ItemKNNCF

from Base.NonPersonalizedRecommender import TopPop

topPop = TopPop(URM_train)
topPop.fit()


from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender_class = ItemKNNCBFRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)


hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine"])
hyperparameters_range_dictionary["normalize"] = Categorical([True, False])
    
    
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train, ICM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


output_folder_path = "result_experiments/"

import os

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
    
    
n_cases = 2
metric_to_optimize = "MAP"

parameterSearch.search(recommender_input_args,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = 1,
                       save_model = "no",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

best_parameters_ItemKNNCBF = search_metadata["hyperparameters_best"]
best_parameters_ItemKNNCBF

itemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM_all)
itemKNNCBF.fit(**best_parameters_ItemKNNCBF)

urm_train


MAP_itemKNNCF_per_group = []
MAP_itemKNNCBF_per_group = []
MAP_pureSVD_per_group = []
MAP_topPop_per_group = []
cutoff = 10

for group_id in range(0, 10):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, average p.len {:.2f}, min {}, max {}".format(group_id, 
        users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))
    
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert = True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users = users_not_in_group)
    
    
    results, _ = evaluator_test.evaluateRecommender(itemKNNCF)
    MAP_itemKNNCF_per_group.append(results[cutoff]["MAP"])
 
    results, _ = evaluator_test.evaluateRecommender(pureSVD)
    MAP_pureSVD_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(itemKNNCBF)
    MAP_itemKNNCBF_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(topPop)
    MAP_topPop_per_group.append(results[cutoff]["MAP"])

    
    

import matplotlib.pyplot as pyplot
%matplotlib inline  

pyplot.plot(MAP_itemKNNCF_per_group, label="itemKNNCF")
pyplot.plot(MAP_itemKNNCBF_per_group, label="itemKNNCBF")
pyplot.plot(MAP_pureSVD_per_group, label="pureSVD")
pyplot.plot(MAP_topPop_per_group, label="topPop")
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()

### The recommendation quality of the three algorithms changes depending on the user profile length

## Tip:
### If an algorithm works best on average, it does not imply it will work best for ALL user types



Group 0, average p.len 0.64, min 0, max 1
Group 1, average p.len 1.00, min 1, max 1
Group 2, average p.len 1.83, min 1, max 2
Group 3, average p.len 2.00, min 2, max 2
Group 4, average p.len 2.00, min 2, max 2
Group 5, average p.len 2.39, min 2, max 3
Group 6, average p.len 3.00, min 3, max 3
Group 7, average p.len 3.00, min 3, max 3
Group 8, average p.len 3.55, min 3, max 4
Group 9, average p.len 4.00, min 4, max 4
TopPopRecommender: URM Detected 143 (1.80 %) cold users.
TopPopRecommender: URM Detected 2076 (8.34 %) cold items.
Iteration No: 1 started. Evaluating function at random point.
ItemKNNCBFRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNCBFRecommender: URM Detected 2076 (8.34 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 83, 'shrink': 499, 'similarity': 'cosine', 'normalize': True}
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 2541.33 column/sec, elapsed time 0.17 min
SearchBayesianSkopt: Config 0 Exc

Traceback (most recent call last):
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 362, in _objective_function
    result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 272, in _evaluate_on_validation
    result_dict, _ = self.evaluator_validation.evaluateRecommender(recommender_instance)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\Base\Evaluation\Evaluator.py", line 245, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\Base\Evaluation\Evaluator.py", line 449, in _run_evaluation_on_selected_users
    return_scores = True
  File "C

Similarity column 25975 ( 100 % ), 2621.95 column/sec, elapsed time 0.17 min
SearchBayesianSkopt: Config 1 Exception. Config: {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': False} - Exception: Traceback (most recent call last):
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 362, in _objective_function
    result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 272, in _evaluate_on_validation
    result_dict, _ = self.evaluator_validation.evaluateRecommender(recommender_instance)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\Base\Evaluation\Evaluator.py", line 245, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object

Traceback (most recent call last):
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 362, in _objective_function
    result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\ParameterTuning\SearchAbstractClass.py", line 272, in _evaluate_on_validation
    result_dict, _ = self.evaluator_validation.evaluateRecommender(recommender_instance)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\Base\Evaluation\Evaluator.py", line 245, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "C:\Users\matte\OneDrive\Desktop\recommender\RecSys_Course_AT_PoliMi\Base\Evaluation\Evaluator.py", line 449, in _run_evaluation_on_selected_users
    return_scores = True
  File "C

Iteration No: 2 ended. Search finished for the next optimal point.
Time taken: 10.1748
Function value obtained: 65504.0000
Current minimum: 65504.0000
SearchBayesianSkopt: Search complete. Best config is None: None

ItemKNNCBFRecommender: URM Detected 143 (1.80 %) cold users.
ItemKNNCBFRecommender: URM Detected 2076 (8.34 %) cold items.


TypeError: fit() argument after ** must be a mapping, not NoneType

In [ ]:
itemKNNCBF.W_sparse

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [24]:
urm_train_validation = urm_train + urm_validation

In [25]:
best_recommender = CFItemKNN()

In [26]:
best_recommender.fit(urm_train_validation,1e-4,10)

Epoch 1 complete in in 24.12 seconds, loss is 9.863E-01. Samples per second 3756.45
Epoch 2 complete in in 23.45 seconds, loss is 9.608E-01. Samples per second 3864.56
Epoch 3 complete in in 23.32 seconds, loss is 9.369E-01. Samples per second 3885.43
Epoch 4 complete in in 22.71 seconds, loss is 9.159E-01. Samples per second 3989.77
Epoch 5 complete in in 23.88 seconds, loss is 8.962E-01. Samples per second 3794.99
Epoch 6 complete in in 23.66 seconds, loss is 8.776E-01. Samples per second 3829.90
Epoch 7 complete in in 23.95 seconds, loss is 8.614E-01. Samples per second 3783.68
Epoch 8 complete in in 23.40 seconds, loss is 8.472E-01. Samples per second 3871.96
Epoch 9 complete in in 23.28 seconds, loss is 8.326E-01. Samples per second 3892.54
Epoch 10 complete in in 22.69 seconds, loss is 8.187E-01. Samples per second 3993.34


In [27]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(best_recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

In [28]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped  #trust MAP (indicatively)

(0.027385597729690892, 0.09178342490649899, 0.04209122439326914, 5638, 2309)

In [29]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [30]:
goodguys

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [31]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

array([3867, 6701, 2385, ..., 6327, 4324, 4004], dtype=int64)

In [32]:
mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

In [33]:
mapping_to_item_id

{0: 10080,
 1: 19467,
 2: 2665,
 3: 7494,
 4: 17068,
 5: 17723,
 6: 18131,
 7: 20146,
 8: 19337,
 9: 21181,
 10: 18736,
 11: 23037,
 12: 477,
 13: 6927,
 14: 10204,
 15: 13707,
 16: 18999,
 17: 19838,
 18: 19851,
 19: 814,
 20: 2754,
 21: 3907,
 22: 4481,
 23: 5581,
 24: 6549,
 25: 7583,
 26: 8849,
 27: 9014,
 28: 9658,
 29: 12914,
 30: 13439,
 31: 16587,
 32: 20127,
 33: 20345,
 34: 21663,
 35: 23895,
 36: 24008,
 37: 24577,
 38: 4649,
 39: 11189,
 40: 13703,
 41: 22592,
 42: 1957,
 43: 3606,
 44: 4102,
 45: 6770,
 46: 7059,
 47: 10716,
 48: 12304,
 49: 14405,
 50: 14846,
 51: 16685,
 52: 21319,
 53: 21950,
 54: 22181,
 55: 24783,
 56: 9427,
 57: 10908,
 58: 19750,
 59: 1199,
 60: 7589,
 61: 8059,
 62: 21260,
 63: 21267,
 64: 24865,
 65: 1920,
 66: 6380,
 67: 12816,
 68: 14100,
 69: 17509,
 70: 21184,
 71: 2154,
 72: 3025,
 73: 3041,
 74: 4532,
 75: 5200,
 76: 5208,
 77: 5998,
 78: 6141,
 79: 6817,
 80: 7409,
 81: 7753,
 82: 8589,
 83: 9163,
 84: 11152,
 85: 11405,
 86: 13274,
 87: 14

In [34]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
    
    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    
    
    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        
        recommendations = recommender.recommend(user_id=mapped_user_id,
                                                at=recommendation_length,
                                                urm_train = urm_train,
                                                remove_seen=True)
        
        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))
        
    return submission

In [35]:
submission = prepare_submission(ratings, users_to_recommend, urm_train_validation, best_recommender)

In [36]:
submission

[(0, [5085, 1447, 23700, 1560, 10444, 25878, 21866, 637, 8887, 13144]),
 (4342, [12061, 5717, 8544, 23127, 13363, 19114, 25071, 24908, 25491, 4620]),
 (5526, [24075, 12061, 9851, 20496, 5717, 4927, 10786, 25693, 773, 9007]),
 (5923, [5717, 19114, 25693, 23127, 24075, 9007, 9851, 25491, 3668, 10428]),
 (149, [12061, 10269, 10786, 21552, 5717, 25675, 9438, 9007, 12056, 25693]),
 (4072, [24075, 12061, 10786, 19114, 4927, 9851, 4811, 23127, 19255, 22359]),
 (6193, [18190, 8544, 13541, 23127, 23536, 11792, 9284, 15145, 12061, 2300]),
 (7105, [3804, 24075, 25491, 19114, 23127, 23457, 3668, 13837, 3469, 9556]),
 (1, [19089, 23600, 19709, 16630, 12409, 19480, 24075, 22121, 8431, 17257]),
 (150, [17723, 20146, 19089, 7494, 19709, 16630, 23189, 8431, 20982, 23600]),
 (183, [19709, 7494, 23481, 23600, 12409, 20146, 24075, 22121, 14895, 11658]),
 (249, [19089, 7494, 11658, 18984, 2533, 8431, 8894, 16630, 20146, 22121]),
 (296, [19089, 25407, 7494, 8894, 16630, 11295, 8097, 23600, 15691, 19709]),
 

In [37]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [38]:
write_submission(submission)

In this lecture we saw the most simple version of Cosine Similarity, where it just includes a shrink factor. There are different optimizations that we can do to it.

Implement TopK Neighbors
When calculating the cosine similarity we used urm.T.dot(urm) to calculate the enumerator. However, depending of the dataset and the number of items, this matrix could not fit in memory. Implemenent a block version, faster than our vector version but that does not use urm.T.dot(urm) beforehand.

Implement Adjusted Cosine 

Implement Dice Similarity

Implement an implicit CF ItemKNN.

Implement a CF UserKNN model